# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
#df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [29]:
	
from pyspark.sql import SparkSession
from pyspark.sql import column as col
from pyspark.sql.types import DoubleType
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_imm =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [28]:
#df_immDesc = pd.read_sas('I94_SAS_Labels_Descriptions.SAS', format='SAS7BDAT')

In [30]:
df_imm = df_imm.selectExpr(["cicid AS  id",
                            "i94yr AS  year",
                            "i94mon AS month",
                            "i94res AS origin_country_id",
                            "i94port AS city_id",
                            "arrdate AS arrival_date",
                            "i94mode AS transport_mode",
                            "depdate AS departure_date",
                            "i94visa AS visa_purpose",
                            "visatype AS visa_type",
                            "biryear AS birth_year",
                            "gender",
                            "admnum AS admin_num",
                            "airline AS airline_code",
                            "fltno AS flight_number"])

In [18]:
df_imm.show(5)

+----+------+-----+-----------------+-------+------------+--------------+--------------+------------+---------+----------+------+--------------+------------+-------------+
|  id|  year|month|origin_country_id|city_id|arrival_date|transport_mode|departure_date|visa_purpose|visa_type|birth_year|gender|     admin_num|airline_code|flight_number|
+----+------+-----+-----------------+-------+------------+--------------+--------------+------------+---------+----------+------+--------------+------------+-------------+
| 6.0|2016.0|  4.0|            692.0|    XXX|     20573.0|          null|          null|         2.0|       B2|    1979.0|  null| 1.897628485E9|        null|         null|
| 7.0|2016.0|  4.0|            276.0|    ATL|     20551.0|           1.0|          null|         3.0|       F1|    1991.0|     M|  3.73679633E9|        null|        00296|
|15.0|2016.0|  4.0|            101.0|    WAS|     20545.0|           1.0|       20691.0|         2.0|       B2|    1961.0|     M|  6.6664318

In [31]:
fname = 'airport-codes_csv.csv'
df_airports = spark.read.csv(fname, header=True)
df_airports = df_airports.selectExpr(["iata_code",
                                      "name",
                                      "type",
                                      "municipality AS city",
                                      "iso_region AS region",
                                      "iso_country AS country",
                                      "continent",
                                      "coordinates",
                                      "gps_code",
                                      "local_code"])

In [20]:
df_airports.show(5)

+---------+--------------------+-------------+------------+------+-------+---------+--------------------+--------+----------+
|iata_code|                name|         type|        city|region|country|continent|         coordinates|gps_code|local_code|
+---------+--------------------+-------------+------------+------+-------+---------+--------------------+--------+----------+
|     null|   Total Rf Heliport|     heliport|    Bensalem| US-PA|     US|       NA|-74.9336013793945...|     00A|       00A|
|     null|Aero B Ranch Airport|small_airport|       Leoti| US-KS|     US|       NA|-101.473911, 38.7...|    00AA|      00AA|
|     null|        Lowell Field|small_airport|Anchor Point| US-AK|     US|       NA|-151.695999146, 5...|    00AK|      00AK|
|     null|        Epps Airpark|small_airport|     Harvest| US-AL|     US|       NA|-86.7703018188476...|    00AL|      00AL|
|     null|Newport Hospital ...|       closed|     Newport| US-AR|     US|       NA| -91.254898, 35.6087|    null|    

In [32]:
df_join = df_imm.join(df_airports, df_imm.city_id==df_airports.iata_code, 'left')

df_imm = df_join.filter(df_join.iata_code.isNotNull())\
                .select([   "id",
                            "year",
                            "month",
                            "origin_country_id",
                            "city_id",
                            "city",
                            "arrival_date",
                            "transport_mode",
                            "departure_date",
                            "visa_purpose",
                            "visa_type",
                            "birth_year",
                            "gender",
                            "admin_num",
                            "airline_code",
                            "flight_number"  ])

df_airports = df_join.filter(df_join.iata_code.isNotNull())\
                     .select(["iata_code",
                              "name",
                              "type",
                              "city",
                              "region",
                              "country",
                              "continent",
                              "coordinates",
                              "gps_code",
                              "local_code"])\
                     .dropDuplicates()
                     

In [26]:
df_imm.count()

2036075

In [33]:
df_airports.count()

255

In [35]:
df_airports.show(255)

+---------+--------------------+--------------+--------------------+------+-------+---------+--------------------+--------+----------+
|iata_code|                name|          type|                city|region|country|continent|         coordinates|gps_code|local_code|
+---------+--------------------+--------------+--------------------+------+-------+---------+--------------------+--------+----------+
|      AGA|  Al Massira Airport|medium_airport|              Agadir|MA-AGD|     MA|       AF|-9.41306972503662...|    GMAD|      null|
|      TRO|       Taree Airport|medium_airport|               Taree|AU-NSW|     AU|       OC|152.514007568, -3...|    YTRE|      null|
|      BEB|   Benbecula Airport|medium_airport|          Balivanich|GB-SCT|     GB|       EU|-7.36278009414672...|    EGPL|      null|
|      ORL|Orlando Executive...| small_airport|             Orlando| US-FL|     US|       NA| -81.332901, 28.5455|    KORL|       ORL|
|      NRN|       Weeze Airport|medium_airport|        

In [2]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = spark.read.csv(fname, header=True)

In [11]:
df_temp.count()

8599212

In [13]:
df_temp.show(5)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



In [10]:
df_temp.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [25]:
df_temp = df_temp.filter((df_temp.dt[6:2]=="04") & (df_temp.Country == "United States"))\
                 .groupBy(df_temp.City,df_temp.Latitude,df_temp.Longitude)\
                 .agg({"AverageTemperature":"avg"})

In [7]:
df_air.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [9]:
df_air.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [4]:
df_join1 = df_imm.join(df_air, df_imm.i94port==df_air.iata_code, 'left')

In [5]:
fname = 'us-cities-demographics.csv'
df_demo = spark.read.csv(fname, header=True, sep=';')

In [16]:
df_demo.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [18]:
df_demo.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

In [6]:
df_join2 = df_join1.join(df_demo, df_join1.municipality==df_demo.City, 'left')

In [16]:
df_join2.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+-------------------+-------+-------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|        coordinates|   City|  State|Median Age|Male Population|Female Population|Total Population|Number of

In [24]:
import pyspark
#spark.createDataFrame(df_temp.dt[0:4].cast(DoubleType()))
#df_temp.filter((df_temp.dt[6:2]=="04") & (df_temp.Country == "United States")).groupBy(df_temp.City,df_temp.Latitude,df_temp.Longitude).agg({"AverageTemperature":"avg"}).show(20)
#df_temp.withColumn("year", df_temp.dt[6:2]).show(10)

+----------------+--------+---------+-----------------------+
|            City|Latitude|Longitude|avg(AverageTemperature)|
+----------------+--------+---------+-----------------------+
|     New Orleans|  29.74N|   90.46W|     20.316677884615384|
|     Kansas City|  39.38N|   93.64W|     12.053695473251038|
|     Springfield|  37.78N|   93.56W|     13.075181069958841|
|    Independence|  39.38N|   93.64W|     12.053695473251038|
|         Fontana|  34.56N|  116.76W|     15.237381818181817|
|     Los Angeles|  34.56N|  118.70W|     13.934545454545448|
|Rancho Cucamonga|  34.56N|  116.76W|     15.237381818181817|
|        Glendale|  32.95N|  112.02W|     19.554320224719106|
|      Louisville|  37.78N|   85.42W|      13.02154615384615|
|   Overland Park|  39.38N|   93.64W|     12.053695473251038|
|   Santa Clarita|  34.56N|  118.70W|     13.934545454545448|
|         Abilene|  32.95N|  100.53W|      16.99996907216495|
|      Greensboro|  36.17N|   79.56W|     14.271480769230775|
|       

In [26]:
df_join3 = df_join2.join(df_temp, df_join2.City==df_temp.City, 'left')
df_join3.show(100)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+----------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+----------+--------+---------+-----------------------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|      Cit

In [27]:
df_join3.count()

8257281

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.